In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils
from tensorflow.keras.preprocessing import image

In [ ]:
def load_dataset():
    # load train and test dataset
    folder_benign_train = '/content/drive/MyDrive/SkinCancer/input/train/benign'
    folder_malignant_train = '/content/drive/MyDrive/SkinCancer/input/train/malignant'
    
    folder_benign_test = '/content/drive/MyDrive/SkinCancer/input/test/benign'
    folder_malignant_test = '/content/drive/MyDrive/SkinCancer/input/test/malignant'

    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

In [ ]:
def load_dataset():
  # load train and test dataset
  folder_benign_train = '/content/drive/MyDrive/SkinCancer/input/data96x96/train/benign'
  folder_malignant_train = '/content/drive/MyDrive/SkinCancer/input/data96x96/train/malignant'

  folder_benign_test = '/content/drive/MyDrive/SkinCancer/input/data96x96/test/benign'
  folder_malignant_test = '/content/drive/MyDrive/SkinCancer/input/data96x96/test/malignant'

  read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

  # Load in training pictures 
  ims_benign = [read(os.path.join(folder_benign_train, filename)) for filename in os.listdir(folder_benign_train)]
  X_benign = np.array(ims_benign, dtype='uint8')
  ims_malignant = [read(os.path.join(folder_malignant_train, filename)) for filename in os.listdir(folder_malignant_train)]
  X_malignant = np.array(ims_malignant, dtype='uint8')

  # Load in testing pictures
  ims_benign = [read(os.path.join(folder_benign_test, filename)) for filename in os.listdir(folder_benign_test)]
  X_benign_test = np.array(ims_benign, dtype='uint8')
  ims_malignant = [read(os.path.join(folder_malignant_test, filename)) for filename in os.listdir(folder_malignant_test)]
  X_malignant_test = np.array(ims_malignant, dtype='uint8')

  # Create labels
  y_benign = np.zeros(X_benign.shape[0])
  y_malignant = np.ones(X_malignant.shape[0])

  y_benign_test = np.zeros(X_benign_test.shape[0])
  y_malignant_test = np.ones(X_malignant_test.shape[0])


  # Merge data 
  trainX = np.concatenate((X_benign, X_malignant), axis = 0)
  trainY = np.concatenate((y_benign, y_malignant), axis = 0)

  testX = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
  testY = np.concatenate((y_benign_test, y_malignant_test), axis = 0)

  # Shuffle data
  s = np.arange(trainX.shape[0])
  np.random.shuffle(s)
  trainX = trainX[s]
  trainY = trainY[s]

  s = np.arange(testX.shape[0])
  np.random.shuffle(s)
  testX = testX[s]
  testY = testY[s]

  trainY = to_categorical(trainY)

  testY = to_categorical(testY)

  return trainX, trainY, testX, testY



train_images, train_labels, test_images, test_labels = load_dataset()

In [ ]:
model_name = "SkinCancerASMALL96x96"

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model('/content/drive/MyDrive/MeineModelle/' + model_name + '.h5')

model.summary()

In [ ]:
img_path = '/content/drive/MyDrive/SkinCancer/input/test/benign/57.jpg'
img = image.load_img(img_path, target_size=(96, 96))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x.astype('float32')
x = x / 255.0

output_data = model.predict(x)

print('TensorFlow Predicted:', output_data)

In [ ]:
filename =  model_name + '_QUAN'

train_images, test_images = train_images / 255.0 , test_images / 255.0

train_images = tf.cast(train_images, tf.float32)
cifar_ds = tf.data.Dataset.from_tensor_slices((train_images)).batch(1)
def representative_dataset():
  for input_value in cifar_ds.take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

# Save the model.
with open( filename + "_int" + '.tflite', 'wb') as f:
  f.write(tflite_model)

print("Done!")

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=filename + "_int" + '.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
img_path = '/content/drive/MyDrive/SkinCancer/input/test/benign/1.jpg'

x = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(x)
x = np.expand_dims(x, axis=0)
x = x.astype('float32')
x = x / 255.0

interpreter.set_tensor(input_details[0]['index'], x)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print('TensorFlow Lite Predicted:', output_data)